<h1>Assignment 1: Decision Tree/Random Forest</h1>
<h2>Kyle Bejel</h2>

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

Read in necessary data for training and testing.****

In [ ]:
train_data = pd.read_csv("../input/cap-4611-spring-21-assignment-1/train.csv")
test_data = pd.read_csv("../input/cap-4611-spring-21-assignment-1/test.csv")

Use head() to peek at data and get a rough idea of what it looks like.

In [ ]:
train_data.head()

In [ ]:
train_data.isna().values.any()

There are no missing values, so there is no need to remove anything.

In [ ]:
X = train_data.drop(['id','Bankrupt'], axis=1)
y = train_data['Bankrupt']
#X = X.drop(['one if net income was negative for the last two year zero otherwise'], axis=1)
#X = X[(np.abs(stats.zscore(X)) <= 3).all(axis=1)]

First, id and bankupt were dropped from the X data because they were not necessary for making predictions; additionally, another variable was dropped because it had 0 variance and was creating dividing errors when calculating z-scores of every cell to check for outliers. Outliers were found, however dropping them created a difference in row length between y and X, and I just decided to leave them in for this reason.

In [ ]:
X.describe()

Using describe() shows us that all of the data fall in the same range [0,1] so there is no need for any normalizing/standardizing.
Then, we split up the training data using train_test_split for training/validation purposes.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 1)

Then, we make and fit our decision tree model (random state 2 gave me slightly better numbers here) and save their predictions in variables.

In [ ]:
tree = DecisionTreeClassifier(random_state=2)
tree.fit(X_train, y_train)
tree_pred = tree.predict(X_valid)
tree_pred2 = tree.predict_proba(X_valid)[:,1]

forest = RandomForestClassifier(n_estimators=1000, class_weight="balanced", criterion="entropy")
forest.fit(X_train, y_train)
forest_pred = forest.predict(X_valid)
forest_pred2 = forest.predict_proba(X_valid)[:,1]

Using those variables, we print the needed metrics and can now check which model performs better. Thepredict_proba() function will work better when getting our roc_auc score for the random forest, so we will make sure to create variables for that as well.

In [ ]:
print("Decision Tree:")
print("acc: ", metrics.accuracy_score(y_valid, tree_pred))
print("roc: ", metrics.roc_auc_score(y_valid, tree_pred2))
print("f1: ", metrics.f1_score(y_valid, tree_pred2))
print("\nRandom Forest:")
print("acc: ", metrics.accuracy_score(y_valid, forest_pred))
print("roc: ", metrics.roc_auc_score(y_valid, forest_pred2))
print("f1: ", metrics.f1_score(y_valid, forest_pred))

We see that the random forest scores better than the decision tree, and thus will use that model. Next, we will output our results to a file titled "submission.csv" that matches the format of the sample given (id column and Bankrupt column).

In [ ]:
out_id = test_data['id']
test_data = test_data.drop('id', axis=1)
#out = pd.DataFrame({'id' : out_id, 'Bankrupt' : forest.predict(test_data)})
out = pd.DataFrame({'id' : out_id, "Bankrupt" : forest.predict_proba(test_data)[:,1]})
out.to_csv('submission.csv', index=False)

In [ ]:
import pickle
pickle_out = open('rf.pickle','wb')
pickle.dump(forest, pickle_out)